In [32]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [33]:
df = pd.read_csv('data/train_adj.csv')

In [34]:
df.ts = pd.to_datetime(df.ts, format = "%Y-%m-%d %H:%M:%S")

In [35]:
sensing_mat = df.sensing_material.unique()
fixed_cols = ['T', 'RH', 'P', 'sin_hour', 'cos_hour', 'sin_ordate','cos_ordate', 'sin_weekday', 'cos_weekday']
input_cols = ['signal_res', 'heater_res', 'heater_volt', 'age']
sens_cols = ['labels'] + [ str(i) for i in range(4)] * 8 + ['ts']
out_cols = sens_cols + fixed_cols
out_ds = pd.DataFrame(columns=out_cols)

In [36]:
df_red = df.drop_duplicates(subset=['ts'])
df_red.index = np.arange(1, len(df_red)+1)
for col in fixed_cols:
    out_ds[col] = df_red[col]
out_ds.ts = df_red.ts

In [37]:
for out_i, (hour, group_hour) in tqdm(enumerate(df.groupby('ts')), total=len(out_ds)):
    labels = []
    col = 1
    for sens_mat, sens_df in group_hour.groupby('sensing_material'):
        labels += [sens_mat, sens_mat]
        for sens_line in sens_df.iterrows():
            for inpu in input_cols:
                out_ds.iat[out_i, col] = sens_line[1][inpu]
                col += 1
    out_ds.at[out_i, 'labels'] = " ".join(labels)

  0%|          | 0/12199 [00:00<?, ?it/s]

In [38]:
df

,Unnamed: 0,ts,sensing_material,signal_res,heater_res,heater_volt,T,RH,P,sin_hour,cos_hour,sin_ordate,cos_ordate,sin_weekday,cos_weekday,year,age
0,0,2021-01-13 08:00:00,ZnOR,7.513813e+07,67.050000,1.431667,26.691667,22.616667,991.958333,0.259013,-0.038092,-0.000596,0.017157,0.816183,-0.373533,2021,0
1,1,2021-01-13 08:00:00,ZnOR,7.495123e+07,66.125000,1.475833,26.691667,22.616667,991.958333,0.259013,-0.038092,-0.000596,0.017157,0.816183,-0.373533,2021,0
2,2,2021-01-13 08:00:00,LaFeO3,7.472385e+07,70.991667,1.843333,26.691667,22.616667,991.958333,0.259013,-0.038092,-0.000596,0.017157,0.816183,-0.373533,2021,0
3,3,2021-01-13 08:00:00,LaFeO3,2.713244e+07,69.975000,1.802500,26.691667,22.616667,991.958333,0.259013,-0.038092,-0.000596,0.017157,0.816183,-0.373533,2021,0
4,4,2021-01-13 08:00:00,WO3,4.990000e+08,64.950000,1.730000,26.691667,22.616667,991.958333,0.259013,-0.038092,-0.000596,0.017157,0.816183,-0.373533,2021,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97587,103787,2022-07-07 23:00:00,WO3,3.285417e+08,142.530000,2.520500,35.941667,29.576667,993.590000,-0.221540,-0.139495,-0.006505,0.015887,0.126669,-0.888615,2022,434
97588,103788,2022-07-07 23:00:00,ZnOR,4.990000e+08,126.821667,3.891167,35.941667,29.576667,993.590000,-0.221540,-0.139495,-0.006505,0.015887,0.126669,-0.888615,2022,540
97589,103789,2022-07-07 23:00:00,ZnOR,4.990000e+08,125.353333,3.896500,35.941667,29.576667,993.590000,-0.221540,-0.139495,-0.006505,0.015887,0.126669,-0.888615,2022,540
97590,103790,2022-07-07 23:00:00,SmFeO3,2.852844e+07,101.286667,3.820000,35.941667,29.576667,993.590000,-0.221540,-0.139495,-0.006505,0.015887,0.126669,-0.888615,2022,174


In [39]:
out_ds.to_csv('data/parallel_x_labels.csv')

In [40]:
sensing_mat

array(['ZnOR', 'LaFeO3', 'WO3', 'ZnO', 'ZnOg', 'STN', 'SmFeO3'],
      dtype=object)